## Libraries und Variablen definieren

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time

In [2]:
# Define variables
blatter_tb = 'https://shop.blatter-papeterie.ch/rubrik.html?id=1616'
blatter_hc = 'https://shop.blatter-papeterie.ch/rubrik.html?id=1622'

## Webscraping

### Webscraping Hyperlinks 

#### Scrapink all Hyperlinks from Blatter Taschenbücher Neuheiten

In [3]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get(blatter_tb)

# data frame
buch_df = pd.DataFrame(columns=['Titel',  
                            'Preis',
                            'details', 
                            'Text'])

# get all links
all_links_tb = []

for page in range(1,4):
    print('scraping page:', page)
    # scrape all <a> tags
    links = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/form/div/div[2]/table/tbody/tr/td/div//a')

    # get the hyperlinks
    links = [link.get_attribute('href') for link in links]

    # remove duplicates
    links = list(dict.fromkeys(links))

    # add links to all_links
    all_links_tb.extend(links)

    #next page
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div/div[3]/span[5]').click()
    time.sleep(1)

driver.quit()

C:\Users\yanni\AppData\Local\Temp\ipykernel_19220\599895588.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


scraping page: 1
scraping page: 2
scraping page: 3


#### Scrapink all Hyperlinks from Blatter Hardcover Neuheiten

In [4]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get(blatter_hc)

# get all links
all_links_hc = []

for page in range(1,4):
    print('scraping page', page)
    # scrape all <a> tags
    links = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/form/div/div[2]/table/tbody/tr/td/div//a')

    # get the hyperlinks
    links = [link.get_attribute('href') for link in links]

    # remove duplicates
    links = list(dict.fromkeys(links))

    # add links to all_links
    all_links_hc.extend(links)

    #next page
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div/div[3]/span[5]').click()
    time.sleep(1)

driver.quit()

C:\Users\yanni\AppData\Local\Temp\ipykernel_19220\2327163302.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


scraping page 1
scraping page 2
scraping page 3


In [5]:
# merge lists
all_links = []
all_links.extend(all_links_tb)
all_links.extend(all_links_hc)

### Webscraping bookdetails

In [6]:
all_links

['https://shop.blatter-papeterie.ch/artikel.html?id=42940145',
 'https://shop.blatter-papeterie.ch/artikel.html?id=39065098',
 'https://shop.blatter-papeterie.ch/artikel.html?id=39191679',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44093859',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44175982',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44176257',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44175609',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44176005',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44175963',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44175995',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44236051',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44176076',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44175697',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44175624',
 'https://shop.blatter-papeterie.ch/artikel.html?id=44156126',
 'https://shop.blatter-papeterie.ch/artikel.html?id=441

In [7]:
'''
In this code snippet we will get the data from the links
'''

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

for book in all_links:
    try:
        driver.get(book)
        time.sleep(3)
        # get the simple data
        try: 
            titel = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[1]/div[1]/span').text
            titel = titel.split('(')[0]
        except: titel = 'nan'

        try: preis = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[1]/div[2]/div/div[3]/table/tbody/tr[2]/td/div').text
        except Exception: preis = 'nan'

        # get the 'Buchbeschreibungen'
        try: text = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[2]').text
        except Exception: text = 'nan'

        # get the 'Buchdetails'
        try: details = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[1]/div[2]/div/div[2]').text
        except Exception: details = 'nan'

    # Fehlermeldung
    except: 
        print('error mit Buch: ', book)
        continue
    

    df_temp = pd.DataFrame({'Titel': titel,  
                    'Preis': preis,
                    'details': details,
                    'Text': text
                    }, index=[0])
    buch_df = pd.concat([buch_df, df_temp], ignore_index=True)

driver.close()

C:\Users\yanni\AppData\Local\Temp\ipykernel_19220\2271888945.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


## Data wrangling details

In [8]:
buch_df

,Titel,Preis,details,Text
0,Der Auftrag des Papstes,CHF 18.50,"Untertitel Historischer Roman\nAutor Panella, ...",Zusammenfassung\n1274: Nach abenteuerlichen Ja...
1,Im Jahr der Flut,CHF 19.90,Untertitel Vier Frauen und ein Jahrhundertbauw...,Zusammenfassung\n1900. Aufbruch in eine neue Z...
2,Kinderklinik Weißensee - Geteilte Träume,CHF 18.50,Untertitel Roman | Die Kinderklinik-Saga geht ...,Zusammenfassung\nDer langersehnte vierte Band ...
3,Nebelstunde,CHF 22.50,Untertitel Kriminalroman - Die SPIEGEL-Bestsel...,Zusammenfassung\nEine Insel im Dunst der Erinn...
4,Mädchenfeuer,CHF 18.50,Untertitel Ein Kommissar-Bark-Krimi\nAutor Jan...,Zusammenfassung\nEin Kinderskelett in einer Hö...
5,Kräheninsel,CHF 23.90,"Untertitel Thriller\nAutor Bjørk, Samuel / Hae...",Zusammenfassung\nMia Krüger hat der Osloer Pol...
6,Dark Elite - Regrets,CHF 22.50,Untertitel Roman - Mit farbig gestaltetem Buch...,"Zusammenfassung\nWie weit würdest du gehen, um..."
7,Ocean View Avenue - Wo deine Träume wahr werden,CHF 16.90,"Untertitel Roman\nAutor Thompson, Ella\nVerlag...",Zusammenfassung\nWillkommen in der Ocean View ...
8,Das Auge Gottes,CHF 16.90,"Untertitel Roman\nAutor Rollins, James / Stöbe...",Zusammenfassung\nSie werfen einen Blick in die...
9,Die Heimkehr,CHF 16.90,"Autor Grisham, John / Reiter, Bea (Übers.) / D...",Zusammenfassung\nErstmals legt der Meister des...


In [16]:
'''
short algorithm description:
1. List all possible keys: These are the categories that precede the actual values in the data string.
2. Create a regular expression: This regular expression is designed to match each key, followed by any characters until the next key is found or until the end of the string.
3. Find all matches in the data: Use the 're.findall' function to apply the regular expression to the data string. 
This returns a list of tuples containing the key and the associated value for each match.
4. Convert the matches into a Dictionnary in the Dataframe.
'''

def extract_details(details):
    # Key words which can be find in the string
    keys = ["Untertitel", "Autor", "Verlag", "Sprache", "Einband", "Erscheinungsjahr", 
            "Seiten", "Artikelnummer", "ISBN", "Auflage", "Reihe", "Reihenbandnummer"]

    # generate a regex which can find all the keys in the string
    regex = r"\b(" + "|".join(keys) + r")\b(.*?)(?=\s*\b" + "|".join(keys) + r"\b|$)"

    # finds all key value pairs in the string
    matches = re.findall(regex, details, re.DOTALL)
    
    # returns a dictionary with key value pairs
    return dict((k.strip(), v.strip()) for k, v in matches)

In [18]:
def data_wrangling_pipeline(df):
    # apply the function to the data frame (create a new column with extracted details)
    df['details_dict'] = df['details'].apply(extract_details)
    # create new dataframe with the details
    details_df = df['details_dict'].apply(pd.Series, dtype= 'object')
    # concat both dataframes
    new_df = pd.concat([buch_df, details_df], axis=1)
    # drop the old details columns
    new_df.drop(['details', 'details_dict'], axis=1, inplace=True)
    return new_df

In [19]:
buch_df = data_wrangling_pipeline(buch_df)

#drop all rows where the title is nan
buch_df = buch_df[buch_df['Titel'] != 'nan']

#drop duplicates
buch_df.drop_duplicates(subset=['Titel'], inplace=True)

# reset index
buch_df.reset_index(drop=True, inplace=True)

In [21]:
buch_df.tail()

,Titel,Preis,Text,Untertitel,Autor,Verlag,Sprache,Einband,Erscheinungsjahr,Seiten,Artikelnummer,ISBN,Auflage,Reihe,Reihenbandnummer
34,Der Pole,CHF 27.90,Zusammenfassung\nIn seiner Künstlernovelle »De...,Roman,"Coetzee, J.M. / Böhnke, Reinhild (Übers.)",Fischer S.,Deutsch,,2023,144 S.,42780660,978-3-10-397501-7,,NaN,NaN
35,Wagner,CHF 23.90,Zusammenfassung\nTrotz höchster Sicherheitsvor...,Thriller,"Skördeman, Gustaf / Alms, Thorsten (Übers.)",Lübbe Paperback,Deutsch,(Kt),2023,512 S.,42888361,978-3-7857-2850-5,1. Aufl. 2023,,03\nSonstiges Ab 16 J.
36,Jahr der Wunder,CHF 35.50,Zusammenfassung\nDer neue Roman von Pulitzer-P...,Roman,"Erdrich, Louise / Schröder, Gesine (Übers.)",,Deutsch,,2023,464 S.,43002826,978-3-351-03980-6,,NaN,NaN
37,Jahrhundertsommer,CHF 30.50,Zusammenfassung\nEin Dorf ist ein Dorf ist ein...,"Roman | Ein beeindruckendes Frauenschicksal, e...","Grünfelder, Alice",DTV,Deutsch,,2023,320 S.,43379272,978-3-423-28345-8,,NaN,NaN
38,So dunkel die Nacht,CHF 30.50,Zusammenfassung\nDer schlimmste Albtraum einer...,Thriller,"Higgins Clark, Mary / Burke, Alafair / Ebnet, ...",Heyne,Deutsch,,2023,320 S.,42939988,978-3-453-27422-8,NaN,NaN,NaN


## Export Dataframes

In [22]:
# save as xlsx
buch_df.to_excel('../data/excelfiles/df_neuheiten_blatter.xlsx', index=False)
# save as feather
buch_df.to_feather('../data/feather/df_neuheiten_blatter.feather')


## Ausblick

##### Weitere Informationen welche ausgelesen werden könnten:

* informationstext zu Autor
* verfügbarkeit
* Verlagsartikelnummer
